In [1]:
import numpy as np
import time

In [27]:
def monteCarlo(n:int, w:np.array) -> None: # 第二版，模拟d-1维单位锥上的均匀分布
    ## 要求 w[-1] < 0
    start = time.time()
    print("start simulating...")
    w = w[:-1]/(-w[-1])+1
    d = len(w)
    rvec = np.random.uniform(low = 0.0, high = 1.0, size = (n, d))
    power = 1/(d-np.arange(d))
    rvec = rvec**power
    for j in range(d-1):
        rvec[:,j+1] *= rvec[:,j]
    for j in range(d-1):
        rvec[:,d-1-j] = rvec[:,d-2-j]-rvec[:,d-1-j]
    rvec[:,0] = 1-rvec[:,0]
    col = np.dot(rvec, w)
    pos = np.sum(col >= 1)/n
    end = time.time()
    print("end simulating")
    print("simulated", n, "points in", end-start, "s")
    print("positive prob =", pos, ", negative prob =", 1-pos)

In [3]:
def standardMonteCarlo(n:int, w:np.array) -> None:
    start = time.time()
    print("start simulating...")
    w = np.hstack((w, w))
    d = len(w)
    rvec = np.random.normal(loc = 0.0, scale = 1.0, size = (n, d))**2
    col = np.dot(rvec, w)
    pos = np.sum(col >= 0)/n
    end = time.time()
    print("end simulating")
    print("simulated", n, "points in", end-start, "s")
    print("positive prob =", pos, ", negative prob =", 1-pos)

In [29]:
standardMonteCarlo(n, w)
monteCarlo(n, w)
monteCarlo2(n, w)

start simulating...
end simulating
simulated 1000000 points in 1.5716049671173096 s
positive prob = 0.406948 , negative prob = 0.593052
start simulating...
end simulating
simulated 1000000 points in 1.5046138763427734 s
positive prob = 0.405719 , negative prob = 0.5942810000000001
start simulating...
end simulating
simulated 1000000 points in 1.7094781398773193 s
positive prob = 0.406011 , negative prob = 0.593989


In [22]:
n = 10**6
w = np.array([1,-1])

In [12]:
n = 10**6
w = np.array([-12, -6, -6, -2, -2, -2, 6, 6, 6, 3, 3, 1])

In [2]:
def monteCarlo2(n:int, w:np.array) -> None: # 第一版，直接模拟d维单位锥的均匀分布
    start = time.time()
    print("start simulating...")
    d = len(w)
    rvec = np.random.uniform(low = 0.0, high = 1.0, size = (n, d))
    power = 1/(d-np.arange(d))
    rvec = rvec**power
    for j in range(d-1):
        rvec[:,j+1] *= rvec[:,j]
    for j in range(d-1):
        rvec[:,d-1-j] = rvec[:,d-2-j]-rvec[:,d-1-j]
    rvec[:,0] = 1-rvec[:,0]
    col = np.dot(rvec, w)
    pos = np.sum(col >= 0)/n
    end = time.time()
    print("end simulating")
    print("simulated", n, "points in", end-start, "s")
    print("positive prob =", pos, ", negative prob =", 1-pos)

In [2]:
def monteCarlo(n:int, m:float, w:np.array) -> None: # 自由度为1的情形
    start = time.time()
    print("start simulating...")
    d = len(w)
    alpha = np.ones(d)*m/2
    rvec = np.random.dirichlet(alpha, size = n)
    col = np.dot(rvec, w)
    pos = np.sum(col >= 0)/n
    end = time.time()
    print("end simulating")
    print("simulated", n, "points in", end-start, "s")
    print("positive prob =", pos, ", negative prob =", 1-pos)

In [10]:
def standardMonteCarlo(n:int, m:int, w:np.array) -> None:
    start = time.time()
    print("start simulating...")
    d = len(w)
    rvec = np.random.gamma(shape = m/2, scale = 0.5, size = (n, d))
    col = np.dot(rvec, w)
    pos = np.sum(col >= 0)/n
    end = time.time()
    print("end simulating")
    print("simulated", n, "points in", end-start, "s")
    print("positive prob =", pos, ", negative prob =", 1-pos)

In [15]:
n = 10**6
m = 3.14
w = np.array([-12, -6, -6, -2, -2, -2, 6, 6, 6, 3, 3, 1])

In [16]:
standardMonteCarlo(n, m, w)
monteCarlo(n, m, w)

start simulating...
end simulating
simulated 1000000 points in 1.0405011177062988 s
positive prob = 0.37854 , negative prob = 0.62146
start simulating...
end simulating
simulated 1000000 points in 1.2039604187011719 s
positive prob = 0.37797 , negative prob = 0.6220300000000001
